In [7]:
import pandas as pd
import os
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import csv
import openpyxl
from multiprocessing import Pool
import tqdm
import time

In [8]:
path="D:\\all_CTI_report"
file_path={}
for res in os.walk(path):
    if "\\txt" == res[0][-4:]:
        p=res[0]
        source=p.replace("D:\\all_CTI_report\\", "").replace("\\txt", "").replace("\\", "_")
        file_path[source]=p
sources=list(file_path.keys())

In [9]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return None

def sent2token(sentence):
    sw=stopwords.words('english')+[i for i in string.hexdigits+string.punctuation]
    tokens = nltk.word_tokenize(sentence)  
    tagged_sent = nltk.pos_tag(tokens)     

    wnl = WordNetLemmatizer()
    lemmas_sent = []
    for tag in tagged_sent:
        wordnet_pos = get_wordnet_pos(tag[1]) or nltk.corpus.wordnet.NOUN
        lemmas_sent.append(wnl.lemmatize(tag[0], pos=wordnet_pos).lower())
        token=[i for i in lemmas_sent if i not in sw]
    return token,lemmas_sent
    

In [ ]:
def txt2token(path, source, txt_name, path_):
    for j in txt_name:
        tokens=[]
        tokens_clean=[]
        with open(path_+"\\"+j, 'rb') as f:
            byte_txt=f.read()
        txt=byte_txt.decode("utf-8")
        txt=txt.replace("\n", " ")
        sent=nltk.sent_tokenize(txt)
        for k in sent:
            token_clean, token=sent2token(k)
            tokens=tokens+token
            tokens_clean=tokens_clean+token_clean
        info=[j, source, len(sent), len(token), len(token_clean)] #, token, sent
        wb = openpyxl.load_workbook(path + '\\CTI_nltk.xlsx', data_only=True)
        s1 = wb.active
        s1.append(info)
        # s2= wb[source]
        # s2.append(info)
        wb.save(path + '\\CTI_nltk.xlsx')
   



In [10]:
info_str = ['txt_name', 'source', '# of sentence', '# of word'] #, 'words', 'sentences'

# with open(path + "\\CTI_nltk.csv", 'w', newline='', encoding='UTF-8') as f:
#     csv_writer = csv.writer(f)
#     csv_writer.writerow(info_str)

wb = openpyxl.Workbook()
s1 = wb.active
s1.append(info_str)
wb.save(path + '\\CTI_nltk.xlsx')

In [6]:
path="D:\\all_CTI_report"
info_str = ['txt_name', 'source', '# of sentence', '# of word', '# of word(w/o stopwords)'] # , 'words', 'sentences'
wb = openpyxl.Workbook()
s1=wb.active
s1.append(info_str)
wb.save(path + '\\CTI_nltk_APT.xlsx')
file_path={}
error_list=[]
for res in os.walk(path):
    if "\\txt" == res[0][-4:]:
        p=res[0]
        source=p.replace("D:\\all_CTI_report\\", "").replace("\\txt", "").replace("\\", "_")
        file_path[source]=p
sources=list(file_path.keys())
for i in sources: 
    start = time.time()
    path_=file_path[i]
    txt_name=os.listdir(path_)
    for j in txt_name:
        tokens=[]
        tokens_clean=[]
        try:
            with open(path_+"\\"+j, 'rb') as f:
                byte_txt=f.read()
            txt=byte_txt.decode("utf-8")
        except:
            error_list.append(path_+"\\"+j)
            continue

        txt=txt.replace("\n", " ")
        sent=nltk.sent_tokenize(txt)
        for k in sent:
            token_clean, token=sent2token(k)
            tokens=tokens+token
            tokens_clean=tokens_clean+token_clean
        info=[j, i, len(sent), len(tokens), len(tokens_clean)]
        wb = openpyxl.load_workbook(path + '\\CTI_nltk_APT.xlsx', data_only=True)
        s1 = wb.active
        s1.append(info)
        wb.save(path + '\\CTI_nltk_APT.xlsx')
    end = time.time()
    print(i,":", end - start, "s")

APTnotes : 897.3435273170471 s
